# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

In [ ]:
# Load the environment variable and Create base models

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if OPENAI_API_KEY:
    print("OPENAI_API_KEY found...")
else:
    print("OPENAI_API_KEY not found...")

MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=OPENAI_API_KEY)
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [ ]:
system_message = "You are an weather assistant helping users to get current temperature in a city"
system_message += "Give short customer answers with location, temperature, humidity and windspeed"
system_message += "always be accurate, if you are not sure, please say you don't know"
system_message += "if you didn't get the correct data or face error in calling a method, explicitly call error in reading data"


In [ ]:
# Chat method 
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create( 
        model = MODEL,
        messages=messages
    )
    return response.choices[0].message.content

# Chat UI using gradio
gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

In [ ]:
# Get live weather data given a city name.
from geopy.geocoders import Nominatim
from typing import Dict, Optional, Tuple

def get_coordinates(city_name: str):
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        print(f"City: {city_name} Latitude: {location.latitude}, Longitude: {location.longitude}")
        return location.latitude, location.longitude
    else:
        print(f"Error in getting geo co-ordinates for {city_name}")
        return None, None



def get_metno_weather(lat: float, lon: float) -> Dict[str, str]:
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    headers = {
        "User-Agent": "weather_app_example/1.0 github.com/yourusername"
    }
    result: Dict[str, str] = {}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        timeseries = data['properties']['timeseries'][0]
        details = timeseries['data']['instant']['details']
        result["Temperature"] = f"{details['air_temperature']}°C"
        result["Humidity"] = f"{details['relative_humidity']}%"
        result["WindSpeed"] = f"{details['wind_speed']} m/s"
        print(f"Temperature: {result["Temperature"]} Humidity: {result["Humidity"]} WindSpeed: {result["WindSpeed"]}")
    else:
        print("Error fetching weather data.")
    return result


def get_weather(city: str) -> Dict[str, str]:
    lat: float = 0.0
    lon: float = 0.0
    lat, lon = get_coordinates(city)
    if lat is None or lon is None:
        print("Error: Could not find coordinates for the city.")
        return {}
    weather_result: Dict[str, str] = get_metno_weather(lat, lon)
    return weather_result


In [ ]:
# Get live weather
city = "chennai"
curr_weather = get_weather(city)

In [ ]:
# There's a particular dictionary structure that's required to describe our function to pass to openai

weather_function = {
    "name": "get_weather",
    "description": "Get the current weather information for a given city, including temperature, humidity, and wind speed.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to get weather information for.",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

# And this is included in a list of tools:
tools = [{"type": "function", "function": weather_function}]

In [ ]:
# Handle tool call
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get("city")
    weather_data = get_weather(city=city)
    response = {
        "role": "tool",
        "content": json.dumps({"city": city, "weather":weather_data}),
        "tool_call_id": tool_call.id
    }
    print(response)
    return response, city


# Chat method 
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create( 
        model = MODEL,
        messages=messages,
        tools=tools
    )
    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        f_response, city = handle_tool_call(message=message)
        messages.append(message)
        messages.append(f_response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content


In [ ]:
gr.ChatInterface(fn=chat, type='messages').launch()

In [ ]:
## Lets add Multimodel

import base64
from io import BytesIO
from PIL import Image

# artist image generator for the given city and given weather condition.
def artistic_image(city: str, weather: str):
    prompt_msg = f"An image representing the {city} city showing important tourist spots at this {weather} condition"
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=prompt_msg,
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))


In [ ]:
city = "mumbai"
curr_weather = get_weather(city)
image = artistic_image(city, json.dumps(curr_weather))
display(image)

## Add audio

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
city = "mumbai"
curr_weather = get_weather(city)

temp = f"Current temperature at {city} city is \
        {curr_weather["Temperature"]} and \
        Humidity is {curr_weather["Humidity"]} \
        and wind speed is {curr_weather["WindSpeed"]}"

talker(temp)

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artistic_image(city, json.dumps(response))
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [ ]:
import gradio as gr
import speech_recognition as sr
import tempfile
import os

# Speech-to-text function using speech_recognition
def transcribe_and_chat(audio, history):
    recognizer = sr.Recognizer()
    with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as temp_audio:
        temp_audio.write(audio)
        temp_audio.flush()
        with sr.AudioFile(temp_audio.name) as source:
            audio_data = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio_data)
                history += [{"role": "user", "content": text}]
                os.unlink(temp_audio.name)  # Clean up temp file
                return "", history
            except sr.UnknownValueError:
                return "", history + [{"role": "assistant", "content": "Sorry, I couldn't understand that."}]
            except sr.RequestError as e:
                return "", history + [{"role": "assistant", "content": f"Speech recognition error: {e}"}]

# UI
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        mic = gr.Audio(sources=["microphone"], type="filepath", label="🎤 Talk to the Assistant")
    with gr.Row():
        clear = gr.Button("Clear")

    # Text input function
    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    # Connect Textbox
    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

    # Connect Microphone
    mic.change(transcribe_and_chat, inputs=[mic, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)